# Подготовка


In [ ]:
#!L
import os
os.chdir('contracode/representjs')

In [334]:
#!L
import os
os.getcwd()

'/home/jupyter/work/resources/contracode/representjs'

In [307]:
print("lel")

lel


In [215]:
#!L:bash
npm install

WARN engine fsevents@2.1.3: wanted: {"node":"^8.16.0 || ^10.6.0 || >=11.0.0"} (current: {"node":"8.10.0","npm":"3.5.2"})
npm WARN optional Skipping failed optional dependency /chokidar/fsevents:
npm WARN notsup Not compatible with your operating system or architecture: fsevents@2.1.3


In [238]:
#!L:bash
python3 scripts/download_data.py

0it [00:00, ?it/s]


In [280]:
#!L
from cloud_ml.storage.api import Storage

# To retrieve client secret:
# 1. Go to link: https://developers.google.com/drive/api/v3/quickstart/python
# 2. Press "Enable the drive API"
# 3. Choose "TVs and limited input devices"
client_secret = "/home/jupyter/work/resources/.secret"

# downloading contents of the remote file into the local one
gdrive = Storage.gdrive(client_secret)
gdrive_file_id = '1Yfgdk37XOEUyBsKnN5-1fguCEmfN3-cq'
dst_path = 'data/pretrain/'
gdrive.get_dir(gdrive_file_id, dst_path)


KeyboardInterrupt



KeyboardInterrupt: 

In [223]:
#!L
from cloud_ml.storage.api import Storage

# To retrieve client secret:
# 1. Go to link: https://developers.google.com/drive/api/v3/quickstart/python
# 2. Press "Enable the drive API"
# 3. Choose "TVs and limited input devices"
client_secret = "/home/jupyter/work/resources/.secret"

# downloading contents of the remote file into the local one
gdrive = Storage.gdrive(client_secret)
gdrive_file_id = '1YhEcc4KykHlJ2tQCrY_hKCsPHFYaIczn'
dst_path = 'data/ft/'
gdrive.get_dir(gdrive_file_id, dst_path)

Done


# Обучение токенизаций


In [303]:
import os
os.getcwd()

'/home/jupyter/work/resources/contracode/representjs'

### Делаем из обучающего датасета .txt корпус

In [2]:
from run_sentencepiece import make_corpus, spm_train

make_corpus(
    input='../data/codesearchnet_javascript/javascript_train_supervised.jsonl.gz',
    output='../data/codesearchnet_javascript/javascript_train_supervised.txt',
    label_mode='docstring'
)

Number of functions: 81487
Example original: function x(options) {
    // Discover referenced files
    const FILES = discoverAndReadFiles(options);
    // Add fake usage files
    options.inlineEntryPoints.forEach((inlineEntryPoint, index) => {
        FILES[`inlineEntryPoint.${index}.ts`] = inlineEntryPoint;
    });
    // Add additional typings
    options.typings.forEach((typing) => {
        const filePath = path.join(options.sourcesRoot, typing);
        FILES[typing] = fs.readFileSync(filePath).toString();
    });
    // Resolve libs
    const RESOLVED_LIBS = {};
    options.libs.forEach((filename) => {
        const filepath = path.join(TYPESCRIPT_LIB_FOLDER, filename);
        RESOLVED_LIBS[`defaultLib:${filename}`] = fs.readFileSync(filepath).toString();
    });
    const compilerOptions = ts.convertCompilerOptionsFromJson(options.compilerOptions, options.sourcesRoot).options;
    const host = new TypeScriptLanguageServiceHost(RESOLVED_LIBS, FILES, compilerOptions);
    retur

2020-12-21 20:53:48.908 | DEBUG    | data.jsonl_dataset:__init__:110 - Loading /home/jupyter/work/resources/contracode/data/codesearchnet_javascript/javascript_train_supervised.jsonl.gz
javascript_train_supervised.jsonl.gz:   4%|▍         | 81487/1843099 [00:23<08:25, 3481.76it/s]
2020-12-21 20:54:12.317 | DEBUG    | data.jsonl_dataset:__init__:126 - Loaded 81487 examples
Writing corpus to txt: 100%|██████████| 81487/81487 [00:20<00:00, 4053.80it/s]


### На .txt корпусе обучаем токенизатор с bpe

In [4]:
spm_train(
    input = '../data/codesearchnet_javascript/javascript_train_supervised.txt',
    model_prefix = "../data/codesearchnet_javascript/csnjs_8k_9995p_bpe_url", 
    vocab_size = 8000,
    character_coverage = 0.9995,
    model_type = "bpe"
)

--input=../data/codesearchnet_javascript/javascript_train_supervised.txt --model_prefix=csnjs_8k_9995p_bpe_url --vocab_size=8000 --character_coverage=0.9995 --model_type=bpe --unk_piece=[UNK] --pad_piece=[PAD] --user_defined_symbols=[CLS],[SEP],[MASK],[EOL],[URL] --hard_vocab_limit=false


### На .txt корпусе обучаем токенизатор с unigram

In [5]:
spm_train(
    input = '../data/codesearchnet_javascript/javascript_train_supervised.txt',
    model_prefix = "../data/codesearchnet_javascript/csnjs_8k_9995p_unigram_url_1", 
    vocab_size = 8000,
    character_coverage = 0.9995,
    model_type = "unigram"
)

--input=../data/codesearchnet_javascript/javascript_train_supervised.txt --model_prefix=../data/codesearchnet_javascript/csnjs_8k_9995p_unigram_url_1 --vocab_size=8000 --character_coverage=0.9995 --model_type=unigram --unk_piece=[UNK] --pad_piece=[PAD] --user_defined_symbols=[CLS],[SEP],[MASK],[EOL],[URL] --hard_vocab_limit=false


### На .txt корпусе обучаем токенизатор с word

In [15]:
spm_train(
    input = '../data/codesearchnet_javascript/javascript_train_supervised.txt',
    model_prefix = "../data/codesearchnet_javascript/csnjs_8k_9995p_word_url", 
    vocab_size = 8000,
    character_coverage = 0.9995,
    model_type = "word"
)

--input=../data/codesearchnet_javascript/javascript_train_supervised.txt --model_prefix=../data/codesearchnet_javascript/csnjs_8k_9995p_word_url --vocab_size=8000 --character_coverage=0.9995 --model_type=word --unk_piece=[UNK] --pad_piece=[PAD] --user_defined_symbols=[CLS],[SEP],[MASK],[EOL],[URL] --hard_vocab_limit=false


### На .txt корпусе обучаем токенизатор с char

In [16]:
spm_train(
    input = '../data/codesearchnet_javascript/javascript_train_supervised.txt',
    model_prefix = "../data/codesearchnet_javascript/csnjs_8k_9995p_char_url", 
    vocab_size = 8000,
    character_coverage = 0.9995,
    model_type = "char"
)

--input=../data/codesearchnet_javascript/javascript_train_supervised.txt --model_prefix=../data/codesearchnet_javascript/csnjs_8k_9995p_char_url --vocab_size=8000 --character_coverage=0.9995 --model_type=char --unk_piece=[UNK] --pad_piece=[PAD] --user_defined_symbols=[CLS],[SEP],[MASK],[EOL],[URL] --hard_vocab_limit=false


# Файн-тьюнинг моделей


### Обучаем модель с токенизацией bpe

In [ ]:
#!L

from main import train

train(
    run_name = "names_ft",
    program_mode = "identity",
    label_mode = "identifier",
    n_decoder_layers = 4,
    subword_regularization_alpha = 0,
    num_epochs = 8,
    save_every = 1,
    batch_size = 8,
    num_workers = 8,
    lr = 1e-4,
    train_filepath = "../data/codesearchnet_javascript/javascript_train_supervised.jsonl.gz",
    eval_filepath = "../data/codesearchnet_javascript/javascript_valid_0.jsonl.gz",
    resume_path = "ckpt_best_bpe.pth",
    spm_filepath = "../data/codesearchnet_javascript/csnjs_8k_9995p_bpe_url.model",
    auto_test = False
)

In [336]:
#!L
from main import test

test(
    checkpoint_file='ckpt_best_bpe.pth',
    test_filepath='../data/codesearchnet_javascript/javascript_test_0.jsonl.gz',
    spm_filepath='../data/codesearchnet_javascript/csnjs_8k_9995p_bpe_url.model',
    program_mode='identity',
    label_mode='identifier',
    num_workers=8,
    batch_size=64,
    limit_dataset_size=-1,
    n_decoder_layers=4,
)

2020-12-22 20:07:30.226 | INFO     | main:test:173 - Test data path ../data/codesearchnet_javascript/javascript_test_0.jsonl.gz
2020-12-22 20:07:30.233 | DEBUG    | data.jsonl_dataset:__init__:110 - Loading /home/jupyter/work/resources/contracode/data/codesearchnet_javascript/javascript_test_0.jsonl.gz
javascript_test_0.jsonl.gz:   0%|          | 840/1843099 [00:00<10:46, 2851.44it/s]

2020-12-22 20:07:30.226 | INFO     | main:test:173 - Test data path ../data/codesearchnet_javascript/javascript_test_0.jsonl.gz
2020-12-22 20:07:30.233 | DEBUG    | data.jsonl_dataset:__init__:110 - Loading /home/jupyter/work/resources/contracode/data/codesearchnet_javascript/javascript_test_0.jsonl.gz
javascript_test_0.jsonl.gz:   0%|          | 6483/1843099 [00:01<08:15, 3703.24it/s]
2020-12-22 20:07:31.991 | DEBUG    | data.jsonl_dataset:__init__:126 - Loaded 4441 examples
2020-12-22 20:07:31.994 | INFO     | main:test:175 - Test dataset size: 59
2020-12-22 20:07:32.151 | INFO     | main:test:190 - Created TransformerModel with 39828480 params


CHECKPOINT ckpt_best_bpe.pth
KEYS odict_keys(['encoder.embedding.weight', 'encoder.pos_encoder.pe', 'encoder.encoder.layers.0.self_attn.in_proj_weight', 'encoder.encoder.layers.0.self_attn.in_proj_bias', 'encoder.encoder.layers.0.self_attn.out_proj.weight', 'encoder.encoder.layers.0.self_attn.out_proj.bias', 'encoder.encoder.layers.0.linear1.weight', 'encoder.encoder.layers.0.linear1.bias', 'encoder.encoder.layers.0.linear2.weight', 'encoder.encoder.layers.0.linear2.bias', 'encoder.encoder.layers.0.norm1.weight', 'encoder.encoder.layers.0.norm1.bias', 'encoder.encoder.layers.0.norm2.weight', 'encoder.encoder.layers.0.norm2.bias', 'encoder.encoder.layers.1.self_attn.in_proj_weight', 'encoder.encoder.layers.1.self_attn.in_proj_bias', 'encoder.encoder.layers.1.self_attn.out_proj.weight', 'encoder.encoder.layers.1.self_attn.out_proj.bias', 'encoder.encoder.layers.1.linear1.weight', 'encoder.encoder.layers.1.linear1.bias', 'encoder.encoder.layers.1.linear2.weight', 'encoder.encoder.layers.1

2020-12-22 20:07:42.266 | INFO     | main:test:208 - Loaded state dict from ckpt_best_bpe.pth


2020-12-22 20:08:15.917 | INFO     | main:test:214 - NLL: 30.34916


KeyboardInterrupt: 

### Обучаем модель с токенизацией unigram

In [ ]:
#!L

from main import train

train(
    run_name = "names_ft",
    program_mode = "identity",
    label_mode = "identifier",
    n_decoder_layers = 4,
    subword_regularization_alpha = 0,
    num_epochs = 8,
    save_every = 1,
    batch_size = 8,
    num_workers = 8,
    lr = 1e-4,
    train_filepath = "../data/codesearchnet_javascript/javascript_train_supervised.jsonl.gz",
    eval_filepath = "../data/codesearchnet_javascript/javascript_valid_0.jsonl.gz",
    resume_path = "ckpt_best_unigram.pth",
    spm_filepath = "../data/codesearchnet_javascript/csnjs_8k_9995p_unigram_url.model",
    auto_test = False
)

In [ ]:
#!L
from main import test

test(
    checkpoint_file='ckpt_best_unigram_latest.pth',
    test_filepath='../data/codesearchnet_javascript/javascript_test_0.jsonl.gz',
    spm_filepath='../data/codesearchnet_javascript/csnjs_8k_9995p_unigram_url.model',
    program_mode='identity',
    label_mode='identifier',
    num_workers=8,
    batch_size=64,
    limit_dataset_size=-1,
    n_decoder_layers=4,
)

test_nll,25.99733
test_nll_avg,30.34916
_step,69
_runtime,54
_timestamp,1608667695


test_nll,▄▅▃▃▅▇▄▆█▃▂▅▄▃▂▃▂▄▄▂▄▄▂▂▃▃▂▄▃▆▂▄▃▂▃▃▂▄▁▃
test_nll_avg,▁▄▄▃▄▅▅▆█▇▆▆▆▅▅▅▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


2020-12-22 20:16:30.695 | INFO     | main:test:173 - Test data path ../data/codesearchnet_javascript/javascript_test_0.jsonl.gz
2020-12-22 20:16:30.701 | DEBUG    | data.jsonl_dataset:__init__:110 - Loading /home/jupyter/work/resources/contracode/data/codesearchnet_javascript/javascript_test_0.jsonl.gz
javascript_test_0.jsonl.gz:   0%|          | 1069/1843099 [00:00<13:49, 2221.81it/s]

2020-12-22 20:16:30.695 | INFO     | main:test:173 - Test data path ../data/codesearchnet_javascript/javascript_test_0.jsonl.gz
2020-12-22 20:16:30.701 | DEBUG    | data.jsonl_dataset:__init__:110 - Loading /home/jupyter/work/resources/contracode/data/codesearchnet_javascript/javascript_test_0.jsonl.gz
javascript_test_0.jsonl.gz:   0%|          | 6483/1843099 [00:01<08:14, 3716.88it/s]
2020-12-22 20:16:32.453 | DEBUG    | data.jsonl_dataset:__init__:126 - Loaded 4441 examples
2020-12-22 20:16:32.457 | INFO     | main:test:175 - Test dataset size: 59
2020-12-22 20:16:32.622 | INFO     | main:test:190 - Created TransformerModel with 39828480 params


CHECKPOINT ckpt_best_unigram_latest.pth
KEYS odict_keys(['encoder.embedding.weight', 'encoder.pos_encoder.pe', 'encoder.encoder.layers.0.self_attn.in_proj_weight', 'encoder.encoder.layers.0.self_attn.in_proj_bias', 'encoder.encoder.layers.0.self_attn.out_proj.weight', 'encoder.encoder.layers.0.self_attn.out_proj.bias', 'encoder.encoder.layers.0.linear1.weight', 'encoder.encoder.layers.0.linear1.bias', 'encoder.encoder.layers.0.linear2.weight', 'encoder.encoder.layers.0.linear2.bias', 'encoder.encoder.layers.0.norm1.weight', 'encoder.encoder.layers.0.norm1.bias', 'encoder.encoder.layers.0.norm2.weight', 'encoder.encoder.layers.0.norm2.bias', 'encoder.encoder.layers.1.self_attn.in_proj_weight', 'encoder.encoder.layers.1.self_attn.in_proj_bias', 'encoder.encoder.layers.1.self_attn.out_proj.weight', 'encoder.encoder.layers.1.self_attn.out_proj.bias', 'encoder.encoder.layers.1.linear1.weight', 'encoder.encoder.layers.1.linear1.bias', 'encoder.encoder.layers.1.linear2.weight', 'encoder.encod

2020-12-22 20:16:42.865 | INFO     | main:test:208 - Loaded state dict from ckpt_best_unigram_latest.pth


2020-12-22 20:17:16.663 | INFO     | main:test:214 - NLL: 19.40902


2020-12-22 20:18:34.847 | INFO     | main:calculate_f1_metric:111 - Took 77.76s to decode 64 identifiers
2020-12-22 20:19:45.032 | INFO     | main:calculate_f1_metric:111 - Took 70.11s to decode 64 identifiers
2020-12-22 20:21:01.724 | INFO     | main:calculate_f1_metric:111 - Took 76.62s to decode 64 identifiers
2020-12-22 20:22:18.760 | INFO     | main:calculate_f1_metric:111 - Took 76.96s to decode 64 identifiers
2020-12-22 20:23:35.640 | INFO     | main:calculate_f1_metric:111 - Took 76.81s to decode 64 identifiers
2020-12-22 20:24:52.426 | INFO     | main:calculate_f1_metric:111 - Took 76.71s to decode 64 identifiers
2020-12-22 20:26:09.382 | INFO     | main:calculate_f1_metric:111 - Took 76.89s to decode 64 identifiers
2020-12-22 20:27:26.245 | INFO     | main:calculate_f1_metric:111 - Took 76.79s to decode 64 identifiers
2020-12-22 20:28:43.038 | INFO     | main:calculate_f1_metric:111 - Took 76.72s to decode 64 identifiers


### Обучаем модель с токенизацией по словам (word)

In [ ]:
#!L

from main import train

train(
    run_name = "names_ft",
    program_mode = "identity",
    label_mode = "identifier",
    n_decoder_layers = 4,
    subword_regularization_alpha = 0,
    num_epochs = 8,
    save_every = 1,
    batch_size = 8,
    num_workers = 8,
    lr = 1e-4,
    train_filepath = "../data/codesearchnet_javascript/javascript_train_supervised.jsonl.gz",
    eval_filepath = "../data/codesearchnet_javascript/javascript_valid_0.jsonl.gz",
    resume_path = "../data/pretrain/ckpt_transformer_pretrain_20k.pth",
    spm_filepath = "../data/codesearchnet_javascript/csnjs_8k_9995p_word_url.model",
    auto_test = False
)

In [ ]:
#!L
from main import test

test(
    checkpoint_file='ckpt_best_bpe_latest.pth',
    test_filepath='../data/codesearchnet_javascript/javascript_test_0.jsonl.gz',
    spm_filepath='../data/codesearchnet_javascript/csnjs_8k_9995p_word_url.model',
    program_mode='identity',
    label_mode='identifier',
    num_workers=8,
    batch_size=64,
    limit_dataset_size=-1,
    n_decoder_layers=4,
)

In [330]:
#!L:bash

cp /home/jupyter/work/pyenv/data/runs/names_ft/ckpt_best.pth ckpt_best_bpe_latest.pth

Process is interrupted.
